# Analysis-4:

* Average delay(Arrival) by Delay Reason by each Airport State

* Analysing the reasons for weather delay in Airport state experiencing most weather delays

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
# Reading data from 'Data' folder

cwd_path = os.getcwd() 
root_dir_path=os.path.abspath(os.path.join(cwd_path, os.pardir)) #To get the parent directory of cwd
root=root_dir_path.replace("\\", "/")
if not os.path.exists(root+"/"+'Data'): #To check for Data folder
    print('Path does not exist.')
else:
    if not os.path.exists(root+"/"+'Data'+"/"+'Flights'): #To check for Flights folder
        print('Path does not exist.')
    else:
        Path=root+"/"+'Data'+"/"+'Flights'
        if os.path.exists(Path+"/"+'flights.csv'): #To check for flights.csv
            Flights = pd.read_csv(Path+"/"+'flights.csv',sep=",")
        if os.path.exists(Path+"/"+'airlines.csv'): #To check for airlines.csv
            Airlines = pd.read_csv(Path+"/"+'airlines.csv',sep=",")
        if os.path.exists(Path+"/"+'airports.csv'): #To check for airports.csv
            Airports = pd.read_csv(Path+"/"+'airports.csv',sep=",")
        if os.path.exists(Path+"/"+'states.csv'): #To check for airports.csv
            States = pd.read_csv(Path+"/"+'states.csv',sep=",")

E:\Installed_softwares\AnacondaPython\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Adding state name to Airports
States = States.rename(columns={'State': 'STATE'})
States_dict = dict(zip(States['Abbreviation'],States['STATE']))
Airports=Airports.dropna(subset=['STATE'])
Airports['STATE_NAME'] = Airports['STATE'].apply(lambda x: States_dict[x])
Airports.head()

IATA_CODE                              AIRPORT         CITY STATE COUNTRY  \
0       ABE  Lehigh Valley International Airport    Allentown    PA     USA   
1       ABI             Abilene Regional Airport      Abilene    TX     USA   
2       ABQ    Albuquerque International Sunport  Albuquerque    NM     USA   
3       ABR            Aberdeen Regional Airport     Aberdeen    SD     USA   
4       ABY   Southwest Georgia Regional Airport       Albany    GA     USA   

   LATITUDE  LONGITUDE    STATE_NAME  
0  40.65236  -75.44040  Pennsylvania  
1  32.41132  -99.68190         Texas  
2  35.04022 -106.60919    New Mexico  
3  45.44906  -98.42183  South Dakota  
4  31.53552  -84.19447       Georgia

In [4]:
#Dropping rows where ORIGIN_AIRPORT is numeric
Flights['ORIGIN_AIRPORT'] = Flights['ORIGIN_AIRPORT'].str.replace('\d+', '')
Flights=Flights[Flights['ORIGIN_AIRPORT']!='']
Flights=Flights.dropna(subset=['ORIGIN_AIRPORT'])

# Considering rows having positive delays(arrival)
Flights=Flights.dropna(subset=['AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY'])

In [5]:
#Adding origin state name to Flights
Airport_State_dict = dict(zip(Airports['IATA_CODE'],Airports['STATE_NAME']))
Flights['ORIGIN_AIRPORT_STATE'] = Flights['ORIGIN_AIRPORT'].apply(lambda x: Airport_State_dict[x])
Flights.head()

YEAR  MONTH  DAY  DAY_OF_WEEK AIRLINE  FLIGHT_NUMBER TAIL_NUMBER  \
27  2015      1    1            4      NK            597      N528NK   
30  2015      1    1            4      NK            168      N629NK   
35  2015      1    1            4      HA             17      N389HA   
50  2015      1    1            4      B6           1030      N239JB   
52  2015      1    1            4      B6           2134      N307JB   

   ORIGIN_AIRPORT DESTINATION_AIRPORT  SCHEDULED_DEPARTURE  \
27            MSP                 FLL                  115   
30            PHX                 ORD                  125   
35            LAS                 HNL                  145   
50            BQN                 MCO                  307   
52            SJU                 MCO                  400   

            ...           ARRIVAL_DELAY  DIVERTED  CANCELLED  \
27          ...                    25.0         0          0   
30          ...                    43.0         0          0   
35          ...                    15.0         0          0   
50          ...                    20.0         0          0   
52          ...                    85.0         0          0   

    CANCELLATION_REASON  AIR_SYSTEM_DELAY  SECURITY_DELAY  AIRLINE_DELAY  \
27                  NaN              25.0             0.0            0.0   
30                  NaN              43.0             0.0            0.0   
35                  NaN               0.0             0.0           15.0   
50                  NaN              20.0             0.0            0.0   
52                  NaN               0.0             0.0           85.0   

    LATE_AIRCRAFT_DELAY  WEATHER_DELAY  ORIGIN_AIRPORT_STATE  
27                  0.0            0.0             Minnesota  
30                  0.0            0.0               Arizona  
35                  0.0            0.0                Nevada  
50                  0.0            0.0          Pennsylvania  
52                  0.0            0.0          Pennsylvania  

[5 rows x 32 columns]

In [6]:
#Flights_Required_Data=pd.DataFrame(Flights[Flights[('YEAR','MONTH')]])
Flights_Required_Data = Flights[['YEAR','MONTH','DAY','ORIGIN_AIRPORT_STATE','AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY']].reset_index(drop=True)
Flights_Required_Data.head()

YEAR  MONTH  DAY ORIGIN_AIRPORT_STATE  AIR_SYSTEM_DELAY  SECURITY_DELAY  \
0  2015      1    1            Minnesota              25.0             0.0   
1  2015      1    1              Arizona              43.0             0.0   
2  2015      1    1               Nevada               0.0             0.0   
3  2015      1    1         Pennsylvania              20.0             0.0   
4  2015      1    1         Pennsylvania               0.0             0.0   

   AIRLINE_DELAY  LATE_AIRCRAFT_DELAY  WEATHER_DELAY  
0            0.0                  0.0            0.0  
1            0.0                  0.0            0.0  
2           15.0                  0.0            0.0  
3            0.0                  0.0            0.0  
4           85.0                  0.0            0.0

In [7]:
# Contribution of each delay category for Total Delay 

df_delay=pd.DataFrame(Flights_Required_Data.groupby('ORIGIN_AIRPORT_STATE')['AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'].sum()).reset_index()
df_delay['TOTAL_DELAY(minutes)']=df_delay['AIR_SYSTEM_DELAY']+df_delay['LATE_AIRCRAFT_DELAY']+df_delay['WEATHER_DELAY']+df_delay['SECURITY_DELAY']+df_delay['AIRLINE_DELAY']
df_delay['WEATHER_DELAY_%']=df_delay['WEATHER_DELAY']/df_delay['TOTAL_DELAY(minutes)']*100
df_delay['AIR_SYSTEM_DELAY_%']=df_delay['AIR_SYSTEM_DELAY']/df_delay['TOTAL_DELAY(minutes)']*100
df_delay['SECURITY_DELAY_%']=df_delay['SECURITY_DELAY']/df_delay['TOTAL_DELAY(minutes)']*100
df_delay['LATE_AIRCRAFT_DELAY%']=df_delay['LATE_AIRCRAFT_DELAY']/df_delay['TOTAL_DELAY(minutes)']*100
df_delay['AIRLINE_DELAY_%']=df_delay['AIRLINE_DELAY']/df_delay['TOTAL_DELAY(minutes)']*100
df_delay = df_delay.rename(columns={'AIR_SYSTEM_DELAY': 'TOTAL_AIR_SYSTEM_DELAY(in minutes)','WEATHER_DELAY':'TOTAL_WEATHER_DELAY(in minutes)','SECURITY_DELAY':'TOTAL_SECURITY_DELAY(in minutes)','LATE_AIRCRAFT_DELAY':'TOTAL_LATE_AIRCRAFT_DELAY(in minutes)','AIRLINE_DELAY':'TOTAL_AIRLINE_DELAY(in minutes)'})
df_delay=df_delay.sort_values('TOTAL_DELAY(minutes)',ascending=False).reset_index(drop=True)
df_delay.head()

ORIGIN_AIRPORT_STATE  TOTAL_AIR_SYSTEM_DELAY(in minutes)  \
0                Texas                           1420143.0   
1           California                           1307528.0   
2             Illinois                           1150295.0   
3              Florida                           1229525.0   
4              Georgia                            679960.0   

   TOTAL_SECURITY_DELAY(in minutes)  TOTAL_AIRLINE_DELAY(in minutes)  \
0                            9889.0                        2505753.0   
1                            8447.0                        2040018.0   
2                            3202.0                        1635775.0   
3                            8429.0                        1486456.0   
4                            1334.0                        1263048.0   

   TOTAL_LATE_AIRCRAFT_DELAY(in minutes)  TOTAL_WEATHER_DELAY(in minutes)  \
0                              2894787.0                         486028.0   
1                              3291856.0                         121585.0   
2                              2016978.0                         493140.0   
3                              2039472.0                         255566.0   
4                              1114569.0                         316769.0   

   TOTAL_DELAY(minutes)  WEATHER_DELAY_%  AIR_SYSTEM_DELAY_%  \
0             7316600.0         6.642812           19.409876   
1             6769434.0         1.796088           19.315175   
2             5299390.0         9.305599           21.706178   
3             5019448.0         5.091516           24.495223   
4             3375680.0         9.383857           20.142905   

   SECURITY_DELAY_%  LATE_AIRCRAFT_DELAY%  AIRLINE_DELAY_%  
0          0.135158             39.564648        34.247506  
1          0.124781             48.628231        30.135725  
2          0.060422             38.060569        30.867232  
3          0.167927             40.631400        29.613934  
4          0.039518             33.017614        37.416106

In [8]:
#Top 5 states experiencing most delays
n=5
df_delay_top_5_states=df_delay[0:n]
df_delay_top_5_states.head()

ORIGIN_AIRPORT_STATE  TOTAL_AIR_SYSTEM_DELAY(in minutes)  \
0                Texas                           1420143.0   
1           California                           1307528.0   
2             Illinois                           1150295.0   
3              Florida                           1229525.0   
4              Georgia                            679960.0   

   TOTAL_SECURITY_DELAY(in minutes)  TOTAL_AIRLINE_DELAY(in minutes)  \
0                            9889.0                        2505753.0   
1                            8447.0                        2040018.0   
2                            3202.0                        1635775.0   
3                            8429.0                        1486456.0   
4                            1334.0                        1263048.0   

   TOTAL_LATE_AIRCRAFT_DELAY(in minutes)  TOTAL_WEATHER_DELAY(in minutes)  \
0                              2894787.0                         486028.0   
1                              3291856.0                         121585.0   
2                              2016978.0                         493140.0   
3                              2039472.0                         255566.0   
4                              1114569.0                         316769.0   

   TOTAL_DELAY(minutes)  WEATHER_DELAY_%  AIR_SYSTEM_DELAY_%  \
0             7316600.0         6.642812           19.409876   
1             6769434.0         1.796088           19.315175   
2             5299390.0         9.305599           21.706178   
3             5019448.0         5.091516           24.495223   
4             3375680.0         9.383857           20.142905   

   SECURITY_DELAY_%  LATE_AIRCRAFT_DELAY%  AIRLINE_DELAY_%  
0          0.135158             39.564648        34.247506  
1          0.124781             48.628231        30.135725  
2          0.060422             38.060569        30.867232  
3          0.167927             40.631400        29.613934  
4          0.039518             33.017614        37.416106

In [161]:
cwd = os.getcwd() #To access the path of current working directory
path_to_write=cwd+'/Analysis-4'

In [165]:
#Writing output to csv
if not os.path.exists(path_to_write+'/'+'Analysis_4_Contribution_of_Various_Delays_in_Top_5_most_delay_experiencing_States.csv'):
    df_delay_top_5_states.to_csv(path_to_write+'/'+'Analysis_4_Contribution_of_Various_Delays_in_Top_5_most_delay_experiencing_States.csv', sep=',',index=False)
else:
    print('File already exists.')

In [166]:
#Plot of Various Delays in top 5 most delay experiencing Airport States

ax1=sns.pointplot(x="ORIGIN_AIRPORT_STATE", y='WEATHER_DELAY_%', data=df_delay_top_5_states, figsize=(15,10),color='Pink',label='Weather_Delay')
ax1=sns.pointplot(x="ORIGIN_AIRPORT_STATE", y='SECURITY_DELAY_%', data=df_delay_top_5_states, figsize=(15,10),color='Cyan',label='Security_Delay')
ax1=sns.pointplot(x="ORIGIN_AIRPORT_STATE", y='AIR_SYSTEM_DELAY_%', data=df_delay_top_5_states, figsize=(15,10),color='Yellow')
ax1=sns.pointplot(x="ORIGIN_AIRPORT_STATE", y='AIRLINE_DELAY_%', data=df_delay_top_5_states, figsize=(15,10),color='Violet',label='Security_Delay')
ax1=sns.pointplot(x="ORIGIN_AIRPORT_STATE", y='LATE_AIRCRAFT_DELAY%', data=df_delay_top_5_states, figsize=(15,10),color='Brown')

plt.title('Trend of Flight Scheduling, Operation  and Cancellation by each Month')
ax1.set(xlabel='Airport State', ylabel='Delay Percentage')

ax1.legend(handles=ax1.lines[::len(df_delay_top_5_states)+1], labels=['WEATHER_DELAY_%', 'SECURITY_DELAY_%','AIR_SYSTEM_DELAY_%','AIRLINE_DELAY_%','LATE_AIRCRAFT_DELAY_%'])
plt.xticks(rotation=20)
plt.title('Contribution of Various Delays in Top 5 most delay experiencing Airport States')
plt.show()

plt.savefig(path_to_write+'/'+'Contribution_of_Various_Delays_in_Top_5_most_delay_experiencing_States.jpeg', format="JPEG")

In [182]:
#Fetching airports with most weather delays
df_weather_delay=df_delay.sort_values('WEATHER_DELAY_%',ascending=False).reset_index(drop=True)
df_weather_delay.head()

ORIGIN_AIRPORT_STATE  TOTAL_AIR_SYSTEM_DELAY(in minutes)  \
0              Georgia                            679960.0   
1             Illinois                           1150295.0   
2             Oklahoma                             71452.0   
3        West Virginia                              8640.0   
4             Arkansas                             82584.0   

   TOTAL_SECURITY_DELAY(in minutes)  TOTAL_AIRLINE_DELAY(in minutes)  \
0                            1334.0                        1263048.0   
1                            3202.0                        1635775.0   
2                             181.0                          89042.0   
3                               0.0                           8229.0   
4                              55.0                          82383.0   

   TOTAL_LATE_AIRCRAFT_DELAY(in minutes)  TOTAL_WEATHER_DELAY(in minutes)  \
0                              1114569.0                         316769.0   
1                              2016978.0                         493140.0   
2                               163071.0                          26640.0   
3                                15507.0                           2573.0   
4                               125026.0                          22009.0   

   TOTAL_DELAY(minutes)  WEATHER_DELAY_%  AIR_SYSTEM_DELAY_%  \
0             3375680.0         9.383857           20.142905   
1             5299390.0         9.305599           21.706178   
2              350386.0         7.603044           20.392367   
3               34949.0         7.362156           24.721737   
4              312057.0         7.052878           26.464396   

   SECURITY_DELAY_%  LATE_AIRCRAFT_DELAY%  AIRLINE_DELAY_%  
0          0.039518             33.017614        37.416106  
1          0.060422             38.060569        30.867232  
2          0.051657             46.540387        25.412545  
3          0.000000             44.370368        23.545738  
4          0.017625             40.065116        26.399985

In [184]:
#Writing output to csv
#Top 10 states experiencing most weather delays
n=10
df_weather_delay_top_10_states=df_weather_delay[0:n]
if not os.path.exists(path_to_write+'/'+'Analysis_4_Weather_delay_rates_in_Top_10_most_delay_experiencing_Airport_States.csv'):
    df_weather_delay_top_10_states.to_csv(path_to_write+'/'+'Analysis_4_Weather_delay_rates_in_Top_10_most_delay_experiencing_Airport_States.csv', sep=',',index=False)
else:
    print('File already exists.')

In [168]:
#Plot of weather delays in top 10 most weather delay experiencing Airport States
plt.figure(figsize=(15,10))
n=10
ax=sns.barplot(x="ORIGIN_AIRPORT_STATE", y="WEATHER_DELAY_%", data=df_weather_delay[0:10],color='blue')
plt.title('Weather delays in Top 10 most delay experiencing Airport States')
ax.set(xlabel='Airport State', ylabel='Weathe Delay Percentage')
plt.xticks(rotation=20)
plt.show()
plt.savefig(path_to_write+'/'+'Weather_delay_rates_in_Top_10_most_delay_experiencing_Airport_States.jpeg', format="JPEG")

In [12]:
#Fetching top state having more weather delay
n=1
df_weather_delay_top_state=df_weather_delay[0:n]
states_selected=[]  
states_selected.append(df_weather_delay['ORIGIN_AIRPORT_STATE'][0:n]) #List to store State names
df_weather_delay_top_state.head()

ORIGIN_AIRPORT_STATE  TOTAL_AIR_SYSTEM_DELAY(in minutes)  \
0              Georgia                            679960.0   

   TOTAL_SECURITY_DELAY(in minutes)  TOTAL_AIRLINE_DELAY(in minutes)  \
0                            1334.0                        1263048.0   

   TOTAL_LATE_AIRCRAFT_DELAY(in minutes)  TOTAL_WEATHER_DELAY(in minutes)  \
0                              1114569.0                         316769.0   

   TOTAL_DELAY(minutes)  WEATHER_DELAY_%  AIR_SYSTEM_DELAY_%  \
0             3375680.0         9.383857           20.142905   

   SECURITY_DELAY_%  LATE_AIRCRAFT_DELAY%  AIRLINE_DELAY_%  
0          0.039518             33.017614        37.416106

In [13]:
# Dataframe with all rows of Flights data for these airport states

from itertools import chain
state=list(chain(*states_selected))

df1=pd.DataFrame(Flights_Required_Data[Flights_Required_Data['ORIGIN_AIRPORT_STATE']==state[0]])
#df2=pd.DataFrame(Flights_Required_Data[Flights_Required_Data['ORIGIN_AIRPORT_STATE']==state[1]])
#df3=pd.DataFrame(Flights_Required_Data[Flights_Required_Data['ORIGIN_AIRPORT_STATE']==state[2]])

frames=[df1]
df_final=pd.concat(frames).reset_index(drop=True)
df_final.head()

YEAR  MONTH  DAY ORIGIN_AIRPORT_STATE  AIR_SYSTEM_DELAY  SECURITY_DELAY  \
0  2015      1    1              Georgia              85.0             0.0   
1  2015      1    1              Georgia               5.0             0.0   
2  2015      1    1              Georgia              17.0             0.0   
3  2015      1    1              Georgia               5.0             0.0   
4  2015      1    1              Georgia              34.0             0.0   

   AIRLINE_DELAY  LATE_AIRCRAFT_DELAY  WEATHER_DELAY  
0            0.0                  0.0            0.0  
1           19.0                  0.0            0.0  
2            0.0                  0.0            0.0  
3           51.0                  0.0            0.0  
4            0.0                  0.0            0.0

In [156]:
#Fetching weather data from Weather Underground API for the year 2015 for the state experiencing most weather delay i.e. Georgia
#Extracting data from Weather API files:

import os
import argparse
import requests
import json
import datetime

#Function to ensure if a directory exists in the path or not;if not create one
def ensure_dir(file_path,directory_name):
    if not os.path.exists(file_path+"/"+directory_name):
        os.mkdir(file_path+"/"+directory_name)
    return file_path+"/"+directory_name

cwd = os.getcwd() #To access the path of current working directory
root_dir_path=os.path.abspath(os.path.join(cwd, os.pardir)) #To get the parent directory of cwd
parent_directory="Data"

basepath=ensure_dir(root_dir_path,parent_directory)

data=[]

#Fetching API key from Environmental Variable
apikey= os.getenv('final_api_key')

#Looping to all the calender days of 2015
import datetime as dt
d = dt.datetime(2015, 1,1)
delta = datetime.timedelta(days=1)
while d <= dt.datetime(2015, 12,31):
    date=str(d) #d will return 2015-01-01 00:00:00
    date_only=date[:10] #To extract date only i.e. 2015-01-01 
    year=date[0:4]
    month=date[5:7]
    day=date[8:10]
    date_required=year+month+date
    
    #Converting month number to month name
    monthinteger=int(month)
    month_name = datetime.date(1900, monthinteger, 1).strftime('%B')
        
    #Requesting data 
    data.append(requests.get('http://api.wunderground.com/api/'+apikey+'/history_'+date_required+'/q/GA/Atlanta.json').json())

    #Creating hierarchy of folder structure with year,month name and day
    
    path1=ensure_dir(basepath,'Weather') #To check for a folder with name 'NYT'
    path2=ensure_dir(path1,'Georgia') #To check for a folder with name 'Community_API'
    path3=ensure_dir(path2,str(year)) #To check for a folder with year
    path4=ensure_dir(path3,str(month_name)) #To check for a folder with month name
    path5=ensure_dir(path4,str(day)) #To check for a folder with day

    with open(path5+'/'+'Georgia'+'_'+str(year)+'_'+str(month_name)+'_'+str(day)+'_'+datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')+'.json', 'w') as outfile:
        json.dump(data, outfile)
            
    d += delta #Adding one day to date

print("Json dumped in folder")


Json dumped in folder


In [79]:
# Preparing Weather API required for Analysis from the API files downloaded:

#To fetch all .json files 

import os
from glob import glob
import glob

cwd_path = os.getcwd() 
root_dir_path=os.path.abspath(os.path.join(cwd_path, os.pardir)) #To get the parent directory of cwd
root=root_dir_path.replace("\\", "/")
start_dir=root+'/data/Weather/' #Parent directory

all_files = []
pattern   = "*.json"

#To fetch all .json files present in parent directory and its child directories
for dir,subdir,file in os.walk(start_dir):
    all_files.extend(glob.glob(os.path.join(dir,pattern))) 

#print(all_files[:2])
print('Total number of files in the start_dir and its root directories:',len(all_files))

Total number of files in the start_dir and its root directories: 365


In [82]:
global row #Global variable for storing row_index of excel
row=1 #Starting index

#Function to fetch all the comments of a file and make an entry of required data in excel; 

def read_file_comment(filename):
    import json
    json_file=open(filename)
    #json_file=open('F:/INFO7374-Python/Final_Project/final/Data/Weather/Georgia/2015/August/01/Georgia_2015_August_01_2017_04_22_17_00_02.json')
    data=json.load(json_file)
    
    #print(data)
    # Setting rows and columns indexes
    global row #Telling function that 'row' is a global variable
    col = 0
    
    number_of_articles=len(data)
    #print('Total number of articles:',number_of_articles)

    for j in range(number_of_articles):
    #daily_summary=data[number_of_articles-1]['history']['dailysummary']
    #print(daily_summary)
    #maxpressurei=data[number_of_articles-1]['history']['dailysummary'][0]['maxpressurei']
    #print(maxpressurei)
        date=data[j]['history']['dailysummary'][0]['date']['year']+'/'+data[j]['history']['dailysummary'][0]['date']['mon']+'/'+data[j]['history']['dailysummary'][0]['date']['mday']
        date_year=data[j]['history']['dailysummary'][0]['date']['year']
        date_month=data[j]['history']['dailysummary'][0]['date']['mon']
        date_day=data[j]['history']['dailysummary'][0]['date']['mday']
        fog=data[j]['history']['dailysummary'][0]['fog']
        tornado=data[j]['history']['dailysummary'][0]['tornado']
        thunder=data[j]['history']['dailysummary'][0]['thunder']
        snow=data[j]['history']['dailysummary'][0]['snow']
        min_temp=data[j]['history']['dailysummary'][0]['mintempm']
        max_temp=data[j]['history']['dailysummary'][0]['maxtempi']
        
        worksheet.write(row, col, date)
        worksheet.write(row, col+1, date_year)
        worksheet.write(row, col+2, date_month)
        worksheet.write(row, col+3, date_day)
        worksheet.write(row, col+4, fog)
        worksheet.write(row, col+5, tornado)
        worksheet.write(row, col+6, thunder)
        worksheet.write(row, col+7, snow)
        worksheet.write(row, col+8, min_temp)
        worksheet.write(row, col+9, max_temp)
    
                
        row+=1
    
        j+=1
        

In [84]:
#Writing the data required for analysis to excel sheet

import xlsxwriter
import os

# Create an excel workbook if not present and add a worksheet.
if not os.path.exists('Weather_Data_for_Analysis.xlsx'):
    workbook = xlsxwriter.Workbook('Weather_Data_for_Analysis.xlsx')
    
    #Adding worksheet to the workbook
    worksheet = workbook.add_worksheet()

    #Adding headers(title) to the columns
    worksheet.write(0, 0, 'Date') 
    worksheet.write(0, 1, 'Year') 
    worksheet.write(0, 2, 'Month') 
    worksheet.write(0, 3, 'Day') 
    worksheet.write(0, 4, 'Fog')
    worksheet.write(0, 5, 'Tornado') 
    worksheet.write(0, 6, 'Thunder') 
    worksheet.write(0, 7, 'Snow') 
    worksheet.write(0, 8, 'Min_Temp')
    worksheet.write(0, 9, 'Max_Temp') 

    #For each file, fetch the articles and make an entry of data required
    for filename in all_files:  
        read_file_comment(filename)
    
    workbook.close()
    print('File has been written.')
    
else:
    print('The file already exists.')

File has been written.


In [89]:
#Analysing the reason for Weather Dealy in the state experiencing most weather delaydf_final=pd.concat(frames).reset_index(drop=True)
df_final_Monthly=pd.DataFrame(df_final.groupby(['MONTH','ORIGIN_AIRPORT_STATE']).mean()['WEATHER_DELAY'])
df_final_Monthly=df_final_Monthly.reset_index()
df_final_Monthly.head()

MONTH ORIGIN_AIRPORT_STATE  WEATHER_DELAY
0      1              Georgia       2.625874
1      2              Georgia       4.407294
2      3              Georgia       2.360668
3      4              Georgia       3.701477
4      5              Georgia       7.384196

In [101]:
#Reading weather excel 
root=cwd_path.replace("\\", "/")
df_weather = pd.read_excel(open(root+'/'+'Weather_Data_for_Analysis.xlsx','rb'), sheetname='Sheet1')
df_weather.head()

Date  Year  Month  Day  Fog  Tornado  Thunder  Snow  Min_Temp  \
0  2015/01/20  2015      1   20    0        0        0     0         4   
1  2015/01/20  2015      1   20    0        0        0     0         4   
2  2015/01/20  2015      1   20    0        0        0     0         4   
3  2015/01/20  2015      1   20    0        0        0     0         4   
4  2015/01/20  2015      1   20    0        0        0     0         4   

   Max_Temp  
0        67  
1        67  
2        67  
3        67  
4        67

In [103]:
df_weather_unique=df_weather.Date.unique()
df_weather_unique.shape

(12,)

In [176]:
df_weather_by_month=pd.DataFrame(df_weather.groupby('Month')['Fog','Tornado','Thunder','Snow','Min_Temp','Max_Temp'].mean()).reset_index()
df_weather_by_month=df_weather_by_month.rename(columns={'Month':'MONTH'})
df_weather_by_month.head()

MONTH  Fog  Tornado  Thunder  Snow  Min_Temp  Max_Temp
0      1    0        0        0     0         4        67
1      2    0        0        0     1       -10        33
2      3    1        0        0     0         8        68
3      4    0        0        1     0        14        76
4      5    0        0        0     0        19        87

In [177]:
#Merging weather data by month with flights data 

df_weather_reason=pd.merge(df_final_Monthly,df_weather_by_month,how='inner')
df_weather_reason.head()

MONTH ORIGIN_AIRPORT_STATE  WEATHER_DELAY  Fog  Tornado  Thunder  Snow  \
0      1              Georgia       2.625874    0        0        0     0   
1      2              Georgia       4.407294    0        0        0     1   
2      3              Georgia       2.360668    1        0        0     0   
3      4              Georgia       3.701477    0        0        1     0   
4      5              Georgia       7.384196    0        0        0     0   

   Min_Temp  Max_Temp  
0         4        67  
1       -10        33  
2         8        68  
3        14        76  
4        19        87

In [178]:
#Month name
Months = {1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'}
df_weather_reason['MONTH_NAME'] = df_weather_reason['MONTH'].apply(lambda x: Months[x])
df_weather_reason.head()

MONTH ORIGIN_AIRPORT_STATE  WEATHER_DELAY  Fog  Tornado  Thunder  Snow  \
0      1              Georgia       2.625874    0        0        0     0   
1      2              Georgia       4.407294    0        0        0     1   
2      3              Georgia       2.360668    1        0        0     0   
3      4              Georgia       3.701477    0        0        1     0   
4      5              Georgia       7.384196    0        0        0     0   

   Min_Temp  Max_Temp MONTH_NAME  
0         4        67    January  
1       -10        33   February  
2         8        68      March  
3        14        76      April  
4        19        87        May

In [179]:
def set_reason(row):
    if row["Fog"] == 1:
        return "Fog"
    elif row["Tornado"] == 1:
        return "Tornado"
    elif row["Thunder"] == 1:
        return "Thunder"
    elif row["Snow"] == 1:
        return "Snow"
    else:
        return "Other"

df_weather_reason = df_weather_reason.assign(Weather_Delay_Reason=df_weather_reason.apply(set_reason, axis=1))
df_weather_reason=pd.DataFrame(df_weather_reason.sort_values('MONTH'))
df_weather_reason=df_weather_reason[['MONTH_NAME','ORIGIN_AIRPORT_STATE','Weather_Delay_Reason','Min_Temp','Max_Temp']]
print(df_weather_reason)

   MONTH_NAME ORIGIN_AIRPORT_STATE Weather_Delay_Reason  Min_Temp  Max_Temp
0     January              Georgia                Other         4        67
1    February              Georgia                 Snow       -10        33
2       March              Georgia                  Fog         8        68
3       April              Georgia              Thunder        14        76
4         May              Georgia                Other        19        87
5        June              Georgia              Thunder        23        89
6        July              Georgia              Thunder        23        93
7      August              Georgia                Other        22        87
8   September              Georgia                Other        17        88
9    November              Georgia                Other         7        66
10   December              Georgia                Other        -1        53


In [170]:
#Writing output to csv
if not os.path.exists(path_to_write+'/'+'Analysis_4_Reasons_for_Weather_Delay_in_State_experiencing_Most_Weather_Delays.csv'):
    df_weather_reason.to_csv(path_to_write+'/'+'Analysis_4_Reasons_for_Weather_Delay_in_State_experiencing_Most_Weather_Delays.csv', sep=',',index=False)
else:
    print('File already exists.')

In [173]:
#Plot of AVERAGE_ARRIVAL_DELAY v/s Airline

plt.figure(figsize=(15,10))
ax1=sns.factorplot(x="MONTH_NAME", y="Min_Temp", hue="Weather_Delay_Reason",data=df_weather_reason,figsize=(15,10))
plt.title('Reasons for Weather Delay in State experiencing Most Weather Delays')
ax1.set(xlabel='Month Name', ylabel='Minimum Temperature in Celcius')
plt.xticks(rotation=20)
plt.show()
plt.savefig(path_to_write+'/'+'Reasons_for_Weather_Delay_in_State_experiencing_Most_Weather_Delays.jpeg', format="JPEG")